In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
data = pd.read_csv('mov.csv')
data.head()

,adulto,orçamento,idioma_original,título_original,visão_geral,popularidade,receita,tempo_de_execução,voto_médio,contagem_de_votos,...,gênero3,elenco,diretor,ano_lançamento,mês_lançamento,dia_lançamento,dia_semana_lançamento,empresa_de_produção,país_de_produção,lucro_filme
0,False,30000000,Inglês,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.95,373554033,81.0,7.7,5415,...,Família,Tom Hanks como Woody (voice); Tim Allen como B...,John Lasseter,1995,10,30,Segunda-feira,Pixar Animation Studios,United States of America,343554033
1,False,65000000,Inglês,Jumanji,When siblings Judy and Peter discover an encha...,17.02,262797249,104.0,6.9,2413,...,Família,Robin Williams como Alan Parrish; Jonathan Hyd...,Joe Johnston,1995,12,15,Sexta-feira,TriStar Pictures,United States of America,197797249
2,False,0,Inglês,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.71,0,101.0,6.5,92,...,NaN,Walter Matthau como Max Goldman; Jack Lemmon c...,Howard Deutch,1995,12,22,Sexta-feira,Warner Bros.,United States of America,0
3,False,16000000,Inglês,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.86,81452156,127.0,6.1,34,...,Romance,Whitney Houston como Savannah 'Vannah' Jackson...,Forest Whitaker,1995,12,22,Sexta-feira,Twentieth Century Fox Film Corporation,United States of America,65452156
4,False,0,Inglês,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.39,76578911,106.0,5.7,173,...,NaN,Steve Martin como George Banks; Diane Keaton c...,Elliot Davis,1995,2,10,Sexta-feira,Sandollar Productions,United States of America,76578911


In [4]:
data.columns

Index(['adulto', 'orçamento', 'idioma_original', 'título_original',
       'visão_geral', 'popularidade', 'receita', 'tempo_de_execução',
       'voto_médio', 'contagem_de_votos', 'palavras_chave', 'gênero1',
       'gênero2', 'gênero3', 'elenco', 'diretor', 'ano_lançamento',
       'mês_lançamento', 'dia_lançamento', 'dia_semana_lançamento',
       'empresa_de_produção', 'país_de_produção', 'lucro_filme'],
      dtype='object')

In [5]:
# Filtrar empresas com mais filmes que 75% do total e calcular a média de votos médios
df_empresas = (data[data['empresa_de_produção'].isin(data['empresa_de_produção'].value_counts().loc[lambda x: x > data['empresa_de_produção'].value_counts().quantile(0.75)].index)]
               .groupby('empresa_de_produção')['voto_médio'].mean().nlargest(10).reset_index(name='Média de Votos Médios'))

# Adicionar a quantidade de filmes produzidos por cada empresa
df_empresas['Quantidade de Filmes'] = data['empresa_de_produção'].value_counts().loc[df_empresas['empresa_de_produção']].values

# Melt do DataFrame para ter as duas barras para cada empresa no gráfico
df_empresas_melt = pd.melt(df_empresas, id_vars=['empresa_de_produção'], 
                            value_vars=['Média de Votos Médios', 'Quantidade de Filmes'],
                            var_name='Tipo', value_name='Valor')

# Criar o gráfico de barras
fig_empresa_producao = px.bar(df_empresas_melt, 
                               x='empresa_de_produção', 
                               y='Valor', 
                               color='Tipo',
                               barmode='group',
                               labels={'empresa_de_produção': 'Empresa de Produção', 
                                       'Valor': 'Valor', 
                                       'Tipo': 'Tipo'},
                               title='Empresa de Produção com os Melhores Votos Médios (com mais filmes que 75% do total)')

# Mostrar o gráfico
fig_empresa_producao.show()


In [6]:
# Filtrar empresas que produziram menos de 5 filmes
empresas_com_menos_de_5_filmes = data['empresa_de_produção'].value_counts()[data['empresa_de_produção'].value_counts() < 5].index

# Remover outliers com base no filtro acima
data_filtrado = data[~data['empresa_de_produção'].isin(empresas_com_menos_de_5_filmes)]

# Calcular a quantidade de filmes que é 75% do total de filmes produzidos
percentil_75_filmes = data_filtrado['empresa_de_produção'].value_counts().quantile(0.75)

# Filtrar empresas que têm produzido mais filmes que 75% do total de filmes produzidos
empresas_com_mais_filmes = data_filtrado['empresa_de_produção'].value_counts()[data_filtrado['empresa_de_produção'].value_counts() > percentil_75_filmes]

# Filtrar empresas que produziram mais de 5 filmes e têm melhores votos médios
empresas_filtradas = empresas_com_mais_filmes.index

# Calcular a média de voto médio por empresa de produção apenas para as empresas selecionadas
media_votos_por_empresa = data_filtrado[data_filtrado['empresa_de_produção'].isin(empresas_filtradas)].groupby('empresa_de_produção')['voto_médio'].mean()

# Obter as 10 empresas com melhores médias de votos médios
top_10_empresas = media_votos_por_empresa.nlargest(10)

# Filtrar o DataFrame original para incluir apenas as 10 empresas com melhores médias de votos médios
data_filtrado_top_10_empresas = data_filtrado[data_filtrado['empresa_de_produção'].isin(top_10_empresas.index)]

# Obter a contagem de filmes por empresa
contagem_filmes_por_empresa = data_filtrado_top_10_empresas['empresa_de_produção'].value_counts().loc[top_10_empresas.index]

# Criar o DataFrame com as informações
df_empresas = pd.DataFrame({'Empresa de Produção': top_10_empresas.index,
                            'Média de Votos Médios': top_10_empresas.values,
                            'Quantidade de Filmes': contagem_filmes_por_empresa.values})

# Melt do DataFrame para ter as duas barras para cada empresa no gráfico
df_empresas_melt = pd.melt(df_empresas, id_vars=['Empresa de Produção'], 
                            value_vars=['Média de Votos Médios', 'Quantidade de Filmes'],
                            var_name='Tipo', value_name='Valor')

# Criar o gráfico de barras para a média de votos médios e quantidade de filmes produzidos
fig_empresas = px.bar(df_empresas_melt, 
                      x='Empresa de Produção', 
                      y='Valor',
                      color='Tipo',
                      barmode='group',
                      labels={'Empresa de Produção': 'Empresa de Produção', 
                              'Valor': 'Valor', 
                              'Tipo': 'Tipo'},
                      title='Média de Votos Médios e Quantidade de Filmes por Empresa de Produção (sem outliers)')

# Mostrar o gráfico
fig_empresas.show()



In [7]:
# Calcular a contagem de filmes por empresa de produção
contagem_filmes_por_empresa = data['empresa_de_produção'].value_counts().nlargest(10)

# Filtrar o DataFrame original para incluir apenas as 10 empresas com mais filmes produzidos
data_top_10_empresas = data[data['empresa_de_produção'].isin(contagem_filmes_por_empresa.index)]

# Calcular a média de voto médio por empresa de produção
media_votos_por_empresa = data_top_10_empresas.groupby('empresa_de_produção')['voto_médio'].mean()

# Ordenar a média de voto médio por empresa de acordo com a ordem das empresas no gráfico de quantidade de filmes
media_votos_por_empresa = media_votos_por_empresa.reindex(contagem_filmes_por_empresa.index)

# Criar o gráfico de barras para a quantidade de filmes produzidos por empresa
fig_quantidade_filmes = go.Figure()
fig_quantidade_filmes.add_trace(go.Bar(x=contagem_filmes_por_empresa.index, y=contagem_filmes_por_empresa.values,
                                       marker_color='blue'))
fig_quantidade_filmes.update_layout(title='Quantidade de Filmes Produzidos por Empresa de Produção',
                                    xaxis_title='Empresa de Produção', yaxis_title='Quantidade de Filmes')

# Criar o gráfico de barras para a média de voto médio por empresa
fig_voto_medio = go.Figure()
fig_voto_medio.add_trace(go.Bar(x=media_votos_por_empresa.index, y=media_votos_por_empresa.values,
                                marker_color='green'))
fig_voto_medio.update_layout(title='Média de Votos Médios por Empresa de Produção',
                             xaxis_title='Empresa de Produção', yaxis_title='Média de Votos Médios',
                             yaxis=dict(range=[0, 10]))

# Mostrar os gráficos
fig_quantidade_filmes.show()
fig_voto_medio.show()



In [8]:
# Calcular a quantidade de filmes produzidos por cada empresa
contagem_filmes_por_empresa2 = data['empresa_de_produção'].value_counts()

# Calcular a média de votos médios por empresa de produção
media_votos_por_empresa = data.groupby('empresa_de_produção')['voto_médio'].mean()

# Filtrar empresas que produziram mais de 5 filmes e têm melhores votos médios
empresas_com_mais_de_5_filmes_e_melhores_votos = media_votos_por_empresa[contagem_filmes_por_empresa2 > 5].nlargest(10)

# Filtrar as 10 empresas com mais filmes produzidos
top_10_empresas_mais_filmes = contagem_filmes_por_empresa2.nlargest(10).index

# Calcular a média de votos médios das 10 empresas com mais filmes produzidos e melhores votos médios
media_votos_top_10_mais_filmes = media_votos_por_empresa[top_10_empresas_mais_filmes]

# Calcular a quantidade de filmes produzidos por cada uma das 10 empresas com mais filmes produzidos
quantidade_filmes_top_10_mais_filmes = contagem_filmes_por_empresa2[top_10_empresas_mais_filmes]

# Criar DataFrame para as empresas com mais filmes produzidos
df_empresas_mais_filmes = pd.DataFrame({
    'Empresa': top_10_empresas_mais_filmes,
    'Voto_medio': media_votos_top_10_mais_filmes.values,
    'Quantidade_de_filmes_produzidos': quantidade_filmes_top_10_mais_filmes.values
})

# Criar DataFrame para as empresas com melhores votos médios e mais de 5 filmes produzidos
df_empresas_melhores_votos_medio = pd.DataFrame({
    'Empresa': empresas_com_mais_de_5_filmes_e_melhores_votos.index,
    'Voto_medio': empresas_com_mais_de_5_filmes_e_melhores_votos.values,
    'Quantidade_de_filmes_produzidos': contagem_filmes_por_empresa2[empresas_com_mais_de_5_filmes_e_melhores_votos.index].values
})

# Exibir os DataFrames
print("DataFrame das empresas com mais filmes produzidos:")
display(df_empresas_mais_filmes)
print("\nDataFrame das empresas com melhores votos médios e mais de 5 filmes produzidos:")
display(df_empresas_melhores_votos_medio)



DataFrame das empresas com mais filmes produzidos:


,Empresa,Voto_medio,Quantidade_de_filmes_produzidos
0,Paramount Pictures,5.935843,996
1,Metro-Goldwyn-Mayer (MGM),5.396827,851
2,Twentieth Century Fox Film Corporation,5.988974,780
3,Warner Bros.,5.587715,757
4,Universal Pictures,6.079310,754
5,Columbia Pictures,5.883217,429
6,Columbia Pictures Corporation,5.763591,401
7,RKO Radio Pictures,5.278621,290
8,United Artists,6.117647,272
9,Walt Disney Pictures,6.232184,261



DataFrame das empresas com melhores votos médios e mais de 5 filmes produzidos:


,Empresa,Voto_medio,Quantidade_de_filmes_produzidos
0,Shaft,8.166667,6
1,Charles Chaplin Productions,7.714286,7
2,Cable Stuff Productions,7.662500,8
3,Studio Ghibli,7.485000,20
4,Netflix,7.314286,21
5,Finos Film,7.300000,10
6,Jigsaw Productions,7.257143,7
7,Arte France,7.250000,12
8,Pixar Animation Studios,7.224138,29
9,HBO Documentary Films,7.190000,10


In [9]:
# Calcular a média da coluna "Voto_medio" de cada DataFrame
media_voto_medio_empresas_mais_filmes = df_empresas_mais_filmes['Voto_medio'].mean()
media_voto_medio_empresas_melhores_votos_medio = df_empresas_melhores_votos_medio['Voto_medio'].mean()

# Calcular a quantidade total de filmes produzidos por cada tipo de empresa
total_filmes_empresas_mais_filmes = df_empresas_mais_filmes['Quantidade_de_filmes_produzidos'].sum()
total_filmes_empresas_melhores_votos_medio = df_empresas_melhores_votos_medio['Quantidade_de_filmes_produzidos'].sum()

# Criar o gráfico de barras
fig = go.Figure()

# Adicionar as barras para a média de votos médios das empresas com mais filmes produzidos
fig.add_trace(go.Bar(x=['Empresas Mais Filmes'], y=[media_voto_medio_empresas_mais_filmes], name='Média Votos Médios'))

# Adicionar as barras para a média de votos médios das empresas com melhores votos médios
fig.add_trace(go.Bar(x=['Empresas Melhores Votos Médio'], y=[media_voto_medio_empresas_melhores_votos_medio], name='Média Votos Médios'))

# Atualizar layout do gráfico
fig.update_layout(title=f'Média de Votos Médios por Empresa\nTotal de Filmes ( Número de filmes Empresas Mais Filmes :{total_filmes_empresas_mais_filmes} / Número de filmes Empresas Melhores Votos Médio:{total_filmes_empresas_melhores_votos_medio})',
                   xaxis_title='Tipo de Empresa',
                   yaxis_title='Média de Votos Médios')

# Mostrar o gráfico
fig.show()


In [10]:
# Filtrar empresas que produziram menos de 5 filmes
empresas_com_menos_de_85_filmes = data['empresa_de_produção'].value_counts()[data['empresa_de_produção'].value_counts() < 85].index

# Remover outliers com base no filtro acima
data_filtrado = data[~data['empresa_de_produção'].isin(empresas_com_menos_de_5_filmes)]

# Calcular a quantidade de filmes produzidos por cada empresa
contagem_filmes_por_empresa = data_filtrado['empresa_de_produção'].value_counts()

# Calcular a média de votos médios por empresa de produção
media_votos_por_empresa = data_filtrado.groupby('empresa_de_produção')['voto_médio'].mean()

# Criar um DataFrame com as informações
df_correlacao85 = pd.DataFrame({'Quantidade de Filmes': contagem_filmes_por_empresa,
                              'Média de Votos Médios': media_votos_por_empresa})



df_correlacao85

,Quantidade de Filmes,Média de Votos Médios
empresa_de_produção,,
120dB Films,7,5.485714
20th Century Fox,6,5.283333
20th Century Fox Home Entertainment,9,5.466667
20th Century Fox Television,19,4.721053
20th Century Pictures,10,3.530000
...,...,...
barefoot films,6,6.283333
uFilm,5,5.180000
ufotable,9,7.122222


In [11]:
# Filtrar empresas que produziram menos de 85 filmes
empresas_com_menos_de_85_filmes = data['empresa_de_produção'].value_counts()[data['empresa_de_produção'].value_counts() < 85].index
data_filtrado_85 = data[~data['empresa_de_produção'].isin(empresas_com_menos_de_85_filmes)]

# Filtrar empresas que produziram menos de 20 filmes
empresas_com_menos_de_20_filmes = data['empresa_de_produção'].value_counts()[data['empresa_de_produção'].value_counts() < 20].index
data_filtrado_20 = data[~data['empresa_de_produção'].isin(empresas_com_menos_de_20_filmes)]

# Calcular média de votos médios por empresa
media_votos_por_empresa = data.groupby('empresa_de_produção')['voto_médio'].mean()

# Criar DataFrames com as informações
df_correlacao_todos = pd.DataFrame({'Quantidade de Filmes': data['empresa_de_produção'].value_counts(), 'Média de Votos Médios': media_votos_por_empresa})
df_correlacao_sem_85 = pd.DataFrame({'Quantidade de Filmes': data_filtrado_85['empresa_de_produção'].value_counts(), 'Média de Votos Médios': data_filtrado_85.groupby('empresa_de_produção')['voto_médio'].mean()})
df_correlacao_sem_20 = pd.DataFrame({'Quantidade de Filmes': data_filtrado_20['empresa_de_produção'].value_counts(), 'Média de Votos Médios': data_filtrado_20.groupby('empresa_de_produção')['voto_médio'].mean()})

# Criar gráficos de dispersão
fig_todos = px.scatter(df_correlacao_todos, x='Quantidade de Filmes', y='Média de Votos Médios', title='Relação entre Quantidade de Filmes e Média de Votos Médios por Empresa (Todos os Dados)')
fig_sem_85 = px.scatter(df_correlacao_sem_85, x='Quantidade de Filmes', y='Média de Votos Médios', title='Relação entre Quantidade de Filmes e Média de Votos Médios por Empresa (Excluindo Empresas com menos de 85 Filmes)')
fig_sem_20 = px.scatter(df_correlacao_sem_20, x='Quantidade de Filmes', y='Média de Votos Médios', title='Relação entre Quantidade de Filmes e Média de Votos Médios por Empresa (Excluindo Empresas com menos de 20 Filmes)')

# Mostrar os gráficos
fig_todos.show()
fig_sem_85.show()
fig_sem_20.show()


In [12]:
# Calcular a correlação entre as duas variáveis
correlacao = df_correlacao_todos['Quantidade de Filmes'].corr(df_correlacao_todos['Média de Votos Médios'])
print(f'A correlação entre a quantidade de filmes produzidos pelas empresas e a média de voto médio é: {correlacao}')
# Calcular a correlação entre as duas variáveis
correlacao1 = df_correlacao_sem_85['Quantidade de Filmes'].corr(df_correlacao_sem_85['Média de Votos Médios'])
print(f'A correlação entre a quantidade de filmes produzidos por empresas com mais de 85 filmes produzidos e a média de voto médio é: {correlacao1}')
correlacao3 = df_correlacao_sem_20['Quantidade de Filmes'].corr(df_correlacao_sem_20['Média de Votos Médios'])
print(f'A correlação entre a quantidade de filmes produzidos por empresas com mais de 20 filmes produzidos e a média de voto médio é: {correlacao3}')

A correlação entre a quantidade de filmes produzidos pelas empresas e a média de voto médio é: 0.01423935819056271
A correlação entre a quantidade de filmes produzidos por empresas com mais de 85 filmes produzidos e a média de voto médio é: -0.4554339696514078
A correlação entre a quantidade de filmes produzidos por empresas com mais de 20 filmes produzidos e a média de voto médio é: -0.07308636628584315


In [13]:
import os
import plotly.io as pio

# Lista dos gráficos
graficos = [fig_empresa_producao, fig_empresas, fig_quantidade_filmes, fig_voto_medio, fig, fig_todos, fig_sem_85, fig_sem_20]

# Pasta de destino
pasta_destino = "graficos_empresas"

# Verificar e criar a pasta se necessário
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

# Lista para armazenar o HTML de cada gráfico
html_graficos = []

# Salvar cada gráfico como HTML na pasta de destino
for i, grafico in enumerate(graficos, start=1):
    nome_arquivo = f"{pasta_destino}/fig_{i}.html"
    pio.write_html(grafico, nome_arquivo)
    html_graficos.append(pio.to_html(grafico, full_html=False))

# Combine o HTML de todos os gráficos em uma única string
html_completo = "\n".join(html_graficos)

# Salvar o arquivo HTML com utf-8
nome_arquivo_html = f"{pasta_destino}/graficos_empresas.html"
with open(nome_arquivo_html, "w", encoding="utf-8") as f:
    f.write(html_completo)

print(f"Gráficos salvos com sucesso em '{nome_arquivo_html}'.")


Gráficos salvos com sucesso em 'graficos_empresas/graficos_empresas.html'.


# Análise Exploratória Detalhada dos Dados de Filmes (Empresas de Produção vs. Voto Médio do Público em Seus Filmes)

## Introdução:

Esta análise aprofundada dos dados de filmes fornece insights valiosos sobre a relação entre quantidade de filmes, média de voto médio e outros fatores relevantes. O objetivo é apresentar os resultados de forma clara e concisa, facilitando a compreensão para leitores que não possuem familiaridade prévia com os dados.

## Metodologia:

- A análise foi realizada em um conjunto de dados contendo informações sobre empresas de cinema, incluindo número de filmes produzidos e média de voto médio.
- Foram utilizados métodos estatísticos como correlação e análise de médias para identificar relações entre as variáveis.
- A análise foi dividida em seções para facilitar a compreensão e a interpretação dos resultados.

## Resultados:

### 1. Quantidade vs. Qualidade:

- A quantidade de filmes não se correlaciona diretamente com a qualidade. Empresas com menor produção podem ter médias de voto médio mais altas.
- Empresas com alta média de voto médio (top 10) tendem a ter uma produção moderada (média de 22 filmes).
- Essa estratégia sugere foco na qualidade, priorizando projetos selecionados e investindo em roteiros, elenco e produção.
- Empresas com alta qualidade podem ter maior reconhecimento, reputação e fidelidade do público.

### 2. Eficiência da Produção:

- Empresas com menor produção (130 filmes) possuem uma média de voto médio significativamente mais alta (7.45) em comparação com empresas de grande porte (5791 filmes, média de 5.82).
- Fatores como foco em nichos específicos, seleção de projetos e atenção à qualidade podem contribuir para essa diferença.

### 3. Tamanho da Amostra e Tendências de Mercado:

- A menor amostra de empresas com 130 filmes exige cautela na interpretação da média de voto médio.
- Empresas com menor produção podem se concentrar em nichos, enquanto empresas com grande produção podem ter estratégias de massa que afetam a qualidade média.
- Diversidade de produção e preferências do público também podem influenciar a média de voto médio.

### 4. Correlação entre Quantidade e Média de Voto:

- A correlação geral entre quantidade de filmes e média de voto médio é muito fraca (0.0142).
- Ao excluir empresas com menos de 85 filmes, encontramos uma correlação negativa moderada (-0.4554).
- Essa correlação indica que, entre empresas com alto volume de produção, existe uma tendência de que quanto mais filmes são feitos, menor é a média de voto médio.
- Empresas com mais de 20 filmes também apresentam uma correlação negativa, porém mais fraca (-0.0731).

### 5. Considerações Importantes:

- Correlação não implica causalidade. Outros fatores podem influenciar a relação entre quantidade e média de voto.
- É necessário um estudo mais aprofundado para entender completamente os fatores que influenciam essa relação.

## Conclusão:

A análise fornece insights valiosos sobre a relação entre quantidade de filmes, média de voto médio e outros fatores. Empresas com menor produção e foco na qualidade podem ter maior sucesso em termos de recepção do público. No entanto, é importante considerar o contexto e outras variáveis para entender completamente as dinâmicas do mercado cinematográfico.
